In [1]:
# Imports
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.model_selection import train_test_split

In [2]:
import numpy as np
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [5]:
checkpoint = torch.load("moco_v1_200ep_pretrain.pth.tar")
checkpoint_dict = checkpoint['state_dict']

for k in list(checkpoint_dict.keys()):
  if k.startswith('module.encoder_q') and not k.startswith('module.encoder_q.fc'):
    checkpoint_dict[k[len("module.encoder_q."):]] = checkpoint_dict[k]
  del checkpoint_dict[k]

model = models.resnet50()
model.load_state_dict(checkpoint_dict, strict=False)
print(model)



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

NUM_CLASSES = 0

def getTrainingSet(dataset_name):
  if dataset_name == 'CIFAR-10':
    print("in cifar-10")
    NUM_CLASSES=10

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                          download=True, transform=transform_train)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                          download=True, transform=transform_test)
    
    
    

    trainset, validset = torch.utils.data.random_split(trainset, 
                                                      [int(len(trainset)*0.8),len(trainset)- 
                                                      int(len(trainset)*0.8)], generator=generator)
    
  elif dataset_name == 'STL10':
    NUM_CLASSES=10
    
    trainset = torchvision.datasets.STL10(root='./data', split='train',
                                          download=True, transform=transform_train)

    testset = torchvision.datasets.STL10(root='./data', split='test',
                                          download=True, transform=transform_train)
    

    trainset, validset = torch.utils.data.random_split(trainset, 
                                                      [int(len(trainset)*0.8),len(trainset)- 
                                                      int(len(trainset)*0.8)], generator=generator)
    

  elif dataset_name == 'Caltech101':
    NUM_CLASSES=101
    !gdown https://drive.google.com/uc?id=1DX_XeKHn3yXtZ18DD7qc1wf-Jy5lnhD5
    !unzip -qq '101_ObjectCategories.zip' 

    PATH = '101_ObjectCategories/'

    transform = transforms.Compose(
      [transforms.CenterCrop(256),
      transforms.Resize((64,64)),
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
    totalset = torchvision.datasets.ImageFolder(PATH, transform=transform_train)

    X, y = zip(*totalset)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, 
                                                      stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, 
                                                    test_size = 0.5, 
                                                    stratify=y_val)

    trainset, validset, testset = list(zip(X_train[:6048], y_train[:6048])), list(zip(X_val, y_val)), list(zip(X_test[:1280], y_test[:1280]))




  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)
  validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size,
                                            shuffle=False,num_workers=2)
  testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)
  return trainset, testset, trainloader, testloader

In [7]:
batch_size = 32
generator=torch.Generator().manual_seed(42) # Can be included for reproducability
train_ds, test_ds, train_loader, test_loader = getTrainingSet("STL10")

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified


In [8]:

class moco(nn.Module):
    def __init__(self, num_labels=10):
        super(moco, self).__init__()
        self.model = model
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(1000, num_labels)
        self.num_labels = num_labels

    def forward(self, images):
        outputs = self.model(images)
        output = self.dropout(outputs)
        logits = self.classifier(output)

        return logits

In [9]:
class moco_compare(nn.Module):
    def __init__(self, num_labels=10):
        super(moco_compare, self).__init__()
        self.model = models.resnet50()
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(1000, num_labels)
        self.num_labels = num_labels

    def forward(self, images):
        outputs = self.model(images)
        output = self.dropout(outputs)
        logits = self.classifier(output)

        return logits

In [10]:
network = moco(10)
#network = moco_compare(10)  
LEARNING_RATE = .0001
optimizer = torch.optim.Adam(network.parameters(), lr=LEARNING_RATE)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
if torch.cuda.is_available():
    network.cuda() 

In [11]:
for batch, image in train_loader:
  print(batch.size())
  print(image)
  print(network(batch.cuda())[0])
  break



torch.Size([32, 3, 32, 32])
tensor([6, 6, 8, 6, 2, 1, 7, 8, 2, 5, 8, 0, 0, 6, 3, 6, 3, 5, 1, 9, 4, 3, 6, 4,
        9, 7, 0, 3, 0, 4, 3, 9])
tensor([-0.0149,  0.0129, -0.0065, -0.0428, -0.0234, -0.0149, -0.0220, -0.0300,
         0.0165,  0.0158], device='cuda:0', grad_fn=<SelectBackward>)


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [12]:
def train(epoch):
  network.train()
  correct_images = 0
  total_images = 0
  training_loss = 0
  losses = 0
  for batch_index, (images, labels) in enumerate(tqdm(train_loader)):
    optimizer.zero_grad()
    images, labels = images.to(device), labels.to(device)
    outputs = network(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    training_loss += loss.item()
    _, predicted = outputs.max(1)
    total_images += labels.size(0)
    correct_images += predicted.eq(labels).sum().item()
  print('Epoch: %d, Loss: %.3f, '
              'Accuracy: %.3f%% (%d/%d)' % (epoch, training_loss/(batch_index+1),
                                       100.*correct_images/total_images, correct_images, total_images))
  return training_loss/(batch_index+1), 100.*correct_images/total_images

In [13]:
def test():
    test_loss = 0
    total_images = 0
    correct_images = 0
    total_loss = 0
    network.eval()
    with torch.no_grad():
      for batch_index, (images, labels) in enumerate(tqdm(test_loader)):
        images, labels = images.to(device), labels.to(device)
        outputs = network(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total_images += labels.size(0)
        correct_images += predicted.eq(labels).sum().item()
        #print(batch_index, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
        #           % (test_loss/(batch_index+1), 100.*correct_images/total_images, correct_images, total_images))
    test_accuracy = 100.*correct_images/total_images
    print("accuracy of test set is",test_accuracy)
    return test_accuracy

In [14]:
# Model

accuracy_test = []
loss = []
train_acc = []
network.to(device)
    
criterion = nn.CrossEntropyLoss()

#RETURN LOSS AFTER EACH EPOCH
for epoch in range(200):
    epoch_loss, epoch_acc = train(epoch)
    loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    accuracy_test.append(test())

100%|██████████| 125/125 [00:06<00:00, 19.45it/s]


Epoch: 0, Loss: 1.887, Accuracy: 31.925% (1277/4000)


100%|██████████| 250/250 [00:05<00:00, 43.83it/s]


accuracy of test set is 39.5875


100%|██████████| 125/125 [00:06<00:00, 19.90it/s]


...

accuracy of test set is 60.45


100%|██████████| 125/125 [00:06<00:00, 18.94it/s]


Epoch: 199, Loss: 0.744, Accuracy: 73.300% (2932/4000)


100%|██████████| 250/250 [00:05<00:00, 41.99it/s]

accuracy of test set is 60.9875


In [15]:
print(loss)
print(train_acc)
print(accuracy_test)

[1.886748556137085, 1.591938530921936, 1.5305738182067872, 1.5125840063095093, 1.46489497756958, 1.44144096326828, 1.4012786588668824, 1.379807300567627, 1.3725113658905028, 1.3714737424850463, 1.3130558485984802, 1.3008307342529297, 1.2930594863891602, 1.2824414300918578, 1.2989414558410644, 1.2819835052490234, 1.2241612706184388, 1.2362469496726989, 1.2502433171272278, 1.2111319017410278, 1.2200733184814454, 1.2318488607406617, 1.1993284029960631, 1.212913980960846, 1.1742140741348266, 1.1795918383598327, 1.1755650796890258, 1.1402935466766357, 1.1991076164245607, 1.173182249546051, 1.1580214657783507, 1.1830520253181458, 1.1533342328071594, 1.1282213726043702, 1.1399291834831238, 1.1474273195266724, 1.1158938465118409, 1.113652166366577, 1.1195157895088195, 1.0983486552238464, 1.125937059879303, 1.0960773077011108, 1.1034858145713806, 1.0544857010841369, 1.090296229839325, 1.0998229746818542, 1.1043711500167848, 1.0720939726829528, 1.1007717366218568, 1.0929085669517518, 1.055333943